# Data preparation and analysis for chat model fine-tuning

This notebook serves as a tool to preprocess and analyze the chat dataset used for fine-tuning a chat model. 
It checks for format errors, provides basic statistics, and estimates token counts for fine-tuning costs.
The method shown here corresponds to the [current fine-tuning method](https://platform.openai.com/docs/guides/fine-tuning) for gpt-3.5-turbo.
See [legacy fine-tuning](https://platform.openai.com/docs/guides/legacy-fine-tuning) for models like babbage-002 and davinci-002.

In [ ]:
! pip install tiktoken python-dotenv

In [0]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

## Data loading

We first load the chat dataset from an [example JSONL file](https://github.com/openai/openai-cookbook/blob/main/examples/data/toy_chat_fine_tuning.jsonl).

In [0]:
# Create the dataset
example_data = [
    {
        "messages": [
            {"role": "system", "content": "You are a happy assistant that puts a positive spin on everything."},
            {"role": "user", "content": "I fell off my bike today."},
            {"role": "assistant", "content": "It's great that you're getting exercise outdoors!"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant who provides concise and accurate information."},
            {"role": "user", "content": "What is the capital of France?"},
            {"role": "assistant", "content": "The capital of France is Paris."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a friendly assistant who helps users with tech problems."},
            {"role": "user", "content": "My laptop won't turn on."},
            {"role": "assistant", "content": "Have you tried holding the power button for 10 seconds? It can sometimes reset the system."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a motivational coach who encourages users to achieve their goals."},
            {"role": "user", "content": "I feel unmotivated to go to the gym today."},
            {"role": "assistant", "content": "Remember, every step you take brings you closer to your fitness goals. You’ve got this!"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a humorous assistant who makes people laugh."},
            {"role": "user", "content": "Tell me a joke about programming."},
            {"role": "assistant", "content": "Why do programmers prefer dark mode? Because light attracts bugs!"}
        ]
    }
]

In [0]:
import json
import tiktoken  # for token counting
import numpy as np
from collections import defaultdict

# Define the dataset path
data_path = "dataset1.json"

# Save the dataset to a JSONL file
with open(data_path, 'w', encoding='utf-8') as f:
    for example in example_data:
        f.write(json.dumps(example) + '\n')

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)


## Format validation

We can perform a variety of error checks to validate that each conversation in the dataset adheres to the format expected by the fine-tuning API. Errors are categorized based on their nature for easier debugging.

1. **Data Type Check**: Checks whether each entry in the dataset is a dictionary (`dict`). Error type: `data_type`.
2. **Presence of Message List**: Checks if a `messages` list is present in each entry. Error type: `missing_messages_list`.
3. **Message Keys Check**: Validates that each message in the `messages` list contains the keys `role` and `content`. Error type: `message_missing_key`.
4. **Unrecognized Keys in Messages**: Logs if a message has keys other than `role`, `content`, `weight`, `function_call`, and `name`. Error type: `message_unrecognized_key`.
5. **Role Validation**: Ensures the `role` is one of "system", "user", or "assistant". Error type: `unrecognized_role`.
6. **Content Validation**: Verifies that `content` has textual data and is a string. Error type: `missing_content`.
7. **Assistant Message Presence**: Checks that each conversation has at least one message from the assistant. Error type: `example_missing_assistant_message`.

The code below performs these checks, and outputs counts for each type of error found are printed. This is useful for debugging and ensuring the dataset is ready for the next steps.


In [0]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

## Token Counting Utilities

Lets define a few helpful utilities to be used in the rest of the notebook.

In [0]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Data Warnings and Token Counts 

With some lightweight analysis we can identify potential issues in the dataset, like missing messages, and provide statistical insights into message and token counts.

1. **Missing System/User Messages**: Counts the number of conversations missing a "system" or "user" message. Such messages are critical for defining the assistant's behavior and initiating the conversation.
2. **Number of Messages Per Example**: Summarizes the distribution of the number of messages in each conversation, providing insight into dialogue complexity.
3. **Total Tokens Per Example**: Calculates and summarizes the distribution of the total number of tokens in each conversation. Important for understanding fine-tuning costs.
4. **Tokens in Assistant's Messages**: Calculates the number of tokens in the assistant's messages per conversation and summarizes this distribution. Useful for understanding the assistant's verbosity.
5. **Token Limit Warnings**: Checks if any examples exceed the maximum token limit (16,385 tokens), as such examples will be truncated during fine-tuning, potentially resulting in data loss.


In [0]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

## Cost Estimation

In this final section, we estimate the total number of tokens that will be used for fine-tuning, which allows us to approximate the cost. It is worth noting that the duration of the fine-tuning jobs will also increase with the token count. 

In [0]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

See https://openai.com/pricing to estimate total costs.

# Ejercicio 2: Generar un Informe de Mensajes Vacíos
Enunciado:
Escribe un código que detecte mensajes donde el contenido (content) esté vacío o solo contenga espacios en blanco. Genera un informe indicando el número total de mensajes vacíos y su distribución por conversación.


In [0]:
# Detectar mensajes vacíos
empty_messages_count = 0
empty_messages_by_conversation = defaultdict(int)

for idx, ex in enumerate(dataset):
    messages = ex["messages"]
    for msg in messages:
        if not msg["content"].strip():
            empty_messages_count += 1
            empty_messages_by_conversation[idx] += 1

print(f"Total de mensajes vacíos: {empty_messages_count}")
print("Distribución por conversación:")
for convo_idx, count in empty_messages_by_conversation.items():
    print(f"Conversación {convo_idx}: {count} mensaje(s) vacío(s)")
#Resultado esperado:
#Si hay mensajes vacíos, el total y su distribución por conversación serán impresos. Ejemplo:


# Ejercicio 4: Filtrar Conversaciones por Token Count
Enunciado:
Escribe un código para filtrar y guardar en un nuevo archivo las conversaciones cuyo número total de tokens sea superior a un umbral especificado (por ejemplo, 500 tokens). Usa la función num_tokens_from_messages.


In [0]:
# Filtrar conversaciones por tokens
threshold = 500
filtered_dataset = [
    ex for ex in dataset if num_tokens_from_messages(ex["messages"]) > threshold
]

# Guardar resultado en archivo
output_path = "../data/filtered_chat_fine_tuning.jsonl"
with open(output_path, 'w', encoding='utf-8') as f:
    for ex in filtered_dataset:
        f.write(json.dumps(ex) + '\n')

print(f"Guardadas {len(filtered_dataset)} conversaciones con más de {threshold} tokens en {output_path}")
##Resultado esperado:
##Las conversaciones con más de 500 tokens se guardarán en un archivo JSONL, y el código imprimirá el número total de conversaciones filtradas.


# Ejercicio 5: Analizar Roles en las Conversaciones
Enunciado:
Escribe un código que cuente la cantidad de mensajes por rol (system, user, assistant) en todo el dataset. Devuelve una tabla resumen con estos conteos.


In [0]:
# Contar roles en el dataset
role_counts = defaultdict(int)

for ex in dataset:
    for msg in ex["messages"]:
        role_counts[msg["role"]] += 1

print("Conteo de roles:")
for role, count in role_counts.items():
    print(f"{role}: {count} mensajes")


# Ejercicio 7: Detectar Conversaciones con Más de un assistant
Enunciado:
Escribe un código para detectar si alguna conversación tiene más de un mensaje con el rol assistant. Devuelve una lista con los índices de las conversaciones problemáticas.


In [0]:
# Detectar múltiples mensajes 'system'
problematic_conversations = []

for idx, ex in enumerate(dataset):
    assistant_messages = [msg for msg in ex["messages"] if msg["role"] == "assistant"]
    if len(assistant_messages) > 1:
        problematic_conversations.append(idx)

print(f"Conversaciones con múltiples mensajes 'assistant': {problematic_conversations}")
#Resultado esperado:
#Si alguna conversación tiene más de un mensaje assistant, se listarán sus índices. Ejemplo:
